### Find Station Venues

Using Foursquare api find the venues within walking distance (1km) of the train stations. Translate the venue sub-categories to parent category.

### Install Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Import Data

In [2]:
Stations_Location = pd.read_csv('Stations.csv')
print(Stations_Location.head())

CategoryKey = pd.read_csv('CategoryKey.csv')
print(CategoryKey.head())

     City     Station   Latitude  Longitude
0  Ottawa       Blair  45.431026 -75.608416
1  Ottawa    Cyrville  45.422745 -75.626372
2  Ottawa  St-Laurent  45.420605 -75.638188
3  Ottawa    Tremblay  45.416933 -75.653348
4  Ottawa     Hurdman  45.412357 -75.664320
                 Parent            categories
0  Arts & Entertainment  Arts & Entertainment
1  Arts & Entertainment          Amphitheater
2  Arts & Entertainment              Aquarium
3  Arts & Entertainment                Arcade
4  Arts & Entertainment           Art Gallery


### Venue Categories


Define Foursquare Credentials

In [3]:
CLIENT_ID = 'IP5SLL30YW00QF4AF4WCUSTRX4FHKJZGGYMRHFXNFN1TCYPM' # your Foursquare ID
CLIENT_SECRET = 'BTECFQLVC11ZLJPRVKA1S3YNYHXOZRS02U5W4ZB4WJP33HWB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IP5SLL30YW00QF4AF4WCUSTRX4FHKJZGGYMRHFXNFN1TCYPM
CLIENT_SECRET:BTECFQLVC11ZLJPRVKA1S3YNYHXOZRS02U5W4ZB4WJP33HWB


Define the Foursquare API

In [4]:
latitude = Stations_Location.iloc[25,2]
longitude = Stations_Location.iloc[25,3]

radius = 800 # in meters; distance travelled in 10 minutes
LIMIT = 1000


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IP5SLL30YW00QF4AF4WCUSTRX4FHKJZGGYMRHFXNFN1TCYPM&client_secret=BTECFQLVC11ZLJPRVKA1S3YNYHXOZRS02U5W4ZB4WJP33HWB&ll=45.421389000000005,-75.69888900000001&v=20180605&radius=800&limit=1000'

Make a request to Foursquare API

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cca0675351e3d1d51a6b850'},
 'response': {'venues': [{'id': '4b72b8f0f964a52072852de3',
    'name': 'Heritage Place',
    'location': {'address': '155 Queen St',
     'lat': 45.42143532601217,
     'lng': -75.69905304443589,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.42143532601217,
       'lng': -75.69905304443589}],
     'distance': 13,
     'postalCode': 'K1P 6E5',
     'cc': 'CA',
     'city': 'Ottawa',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['155 Queen St', 'Ottawa ON K1P 6E5', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d130941735',
      'name': 'Building',
      'pluralName': 'Buildings',
      'shortName': 'Building',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1556743797',
    'hasPerk': False},
   {'id': '5c1d541b180b91003733b977',
    'name': 'Queen St Fare',
    'loc

Extract Categories From Venues

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

__Covert to Pandas dataframe__

In [7]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Heritage Place,Building,45.421435,-75.699053
1,Queen St Fare,Bar,45.420948,-75.699370
2,131 Queen Street,Office,45.421613,-75.698123
3,TD Canada Trust,Bank,45.421207,-75.698347
4,Oracle Ottawa,Office,45.421125,-75.698931


How many venues were returned

In [8]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

158 venues were returned by Foursquare.


In [9]:
nearby_venues_parent = nearby_venues.merge(CategoryKey, on = 'categories')
nearby_venues_parent.head()

,name,categories,lat,lng,Parent
0,Heritage Place,Building,45.421435,-75.699053,Professional & Other Places
1,TD Tower,Building,45.421216,-75.698441,Professional & Other Places
2,HSBC,Building,45.421440,-75.698411,Professional & Other Places
3,Sun Life Financial Centre,Building,45.420793,-75.699438,Professional & Other Places
4,Poulin Building,Building,45.421613,-75.699102,Professional & Other Places


In [10]:
nearby_venues_parent.groupby(['Parent']).size()

Parent
Arts & Entertainment            3
Food                           28
Nightlife Spot                  5
Outdoors & Recreation           6
Professional & Other Places    68
Residence                       1
Shop & Service                 34
Travel & Transport              5
dtype: int64